In [18]:
from qiskit_ibm_runtime import QiskitRuntimeService

token = '4c635c9b66b0ab2f37ce29d7bc33edc66f815847092ba236aa79748304b2cd4bbba6c35d8a4a6adc9b579b1990cfa8066f5220cbdc29044dc9bd59f1c1e1ea1b'
instance = 'ibm-q/open/main'

service = QiskitRuntimeService(
        channel='ibm_quantum',
        instance=instance,
        token=token
)

In [65]:
from qiskit import QuantumCircuit, ClassicalRegister

def create_quantum_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits)
    # Apply Hadamard gate to each qubit
    for i in range(num_qubits):
        qc.h(i)
    # Apply CNOT gate between each pair of qubits
    for i in range(num_qubits - 1):
        qc.cx(i, i + 1)
    # Add classical registers and measurements
    cr = ClassicalRegister(num_qubits)
    qc.add_register(cr)
    qc.measure(range(num_qubits), range(num_qubits))
    return qc

# Specify the number of qubits
num_qubits = 5  # Change this value as needed
qc = create_quantum_circuit(num_qubits)

print(qc)

      ┌───┐          ┌─┐                      
 q_0: ┤ H ├──■───────┤M├──────────────────────
      ├───┤┌─┴─┐     └╥┘     ┌─┐              
 q_1: ┤ H ├┤ X ├──■───╫──────┤M├──────────────
      ├───┤└───┘┌─┴─┐ ║      └╥┘     ┌─┐      
 q_2: ┤ H ├─────┤ X ├─╫───■───╫──────┤M├──────
      ├───┤     └───┘ ║ ┌─┴─┐ ║      └╥┘┌─┐   
 q_3: ┤ H ├───────────╫─┤ X ├─╫───■───╫─┤M├───
      ├───┤           ║ └───┘ ║ ┌─┴─┐ ║ └╥┘┌─┐
 q_4: ┤ H ├───────────╫───────╫─┤ X ├─╫──╫─┤M├
      └───┘           ║       ║ └───┘ ║  ║ └╥┘
c2: 5/════════════════╩═══════╩═══════╩══╩══╩═
                      0       1       2  3  4 


In [66]:
print(service.instances())
print(service.backends())

['ibm-q/open/main']
[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_sherbrooke')>, <IBMBackend('ibm_kyiv')>]


In [67]:
from qiskit_ibm_runtime import Session, Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import state_fidelity
from qiskit_ibm_runtime import RuntimeJob

# List of backends to iterate through
# Get the list of available backends
#backends = ['ibm_brisbane', 'ibm_sherbrooke', 'ibm_kyiv']
#backends =service.backends()
#print(backends)

backend_names = ['ibm_brisbane', 'ibm_sherbrooke', 'ibm_kyiv']

# Ideal state vector (assuming the ideal backend is the first one in the list)
backend = service.backend(backend_names[0])
print(backend)
pass_Manager = generate_preset_pass_manager(optimization_level=1, backend=backend)

# Run it thru pass_Manager transpiler (post 3/2024)
qc_transpiled = pass_Manager.run(qc)

# Run the Sampler job
sampler = Sampler(mode=backend)
job = sampler.run(pubs=[qc_transpiled])
print(job)
job_id = job.job_id()
print(job_id)

<IBMBackend('ibm_brisbane')>
<RuntimeJobV2('cxwd5pe4a290008xxd80', 'sampler')>
cxwd5pe4a290008xxd80


In [73]:
job_id = 'cxwd5pe4a290008xxd80' #'cxwc3r70827000847rj0' 'cxwc9cdy1ae0008na54g' 'cxwce280827000847sc0'
# Get the result
job = service.job(job_id)
print(job)

result = job.result()
print(result)

print(
    f"The result of the submitted job had {len(result)} PUB and has a value:\n {result}\n"
)
print(
    f"The associated PubResult of this Sampler job has the following DataBins:\n {result[0].data}\n"
)
print(f"It has a key-value pair dict: \n{result[0].data.items()}\n")
cr = qc.cregs[0]
print(f"The classical register of the circuit is: {cr}")
print(f"And the raw data can be converted to a bitstring-count format: \n{result[0].data.c2.get_counts()}")

<RuntimeJobV2('cxwd5pe4a290008xxd80', 'sampler')>
PrimitiveResult([SamplerPubResult(data=DataBin(c2=BitArray(<shape=(), num_shots=4096, num_bits=5>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([SliceSpan(<start='2025-01-04 06:19:08', stop='2025-01-04 06:22:09', size=4096>)])}, 'version': 2})
The result of the submitted job had 1 PUB and has a value:
 PrimitiveResult([SamplerPubResult(data=DataBin(c2=BitArray(<shape=(), num_shots=4096, num_bits=5>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([SliceSpan(<start='2025-01-04 06:19:08', stop='2025-01-04 06:22:09', size=4096>)])}, 'version': 2})

The associated PubResult of this Sampler job has the following DataBins:
 DataBin(c2=BitArray(<shape=(), num_shots=4096, num_bits=5>))

It has a key-value pair dict: 
dict_items([('c2', BitArray(<shape=(), num_shots=4096, num_bits=5>))])

The classical register of the circuit is: ClassicalRegiste

In [ ]:



# Ideal state vector (assuming the ideal backend is the first one in the list)
ideal_backend = backends[0]
print(f"Ideal backend: {ideal_backend}")
with Session(backend=ideal_backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run(pubs=[qc])
    ideal_result = job.result()
    ideal_statevector = ideal_result.get_statevector()

# Store results
results = []

# Iterate through the backends and compare results
for backend_name in backends:
    with Session(service=service, backend=backend_name) as session:
        sampler = Sampler(mode=session)
        job = sampler.run(pubs=qc)
        result = job.result()
        prepared_statevector = result.get_statevector()
        
        # Calculate fidelity
        fidelity = state_fidelity(ideal_statevector, prepared_statevector)
        results.append((backend_name, fidelity))

# Display results
import pandas as pd

df = pd.DataFrame(results, columns=['Backend', 'Fidelity'])
print(df)

In [ ]:
import matplotlib.pyplot as plt

# Display the results as a table
print(df)

# Plot the results
plt.figure(figsize=(10, 6))
plt.bar(df['Backend'], df['Fidelity'], color='blue')
plt.xlabel('Backend')
plt.ylabel('Fidelity')
plt.title('Fidelity of Quantum Circuit Execution on Different Backends')
plt.show()